# 🏦 Credit Risk Scorecard — Notebook 3: IFRS 9 Expected Credit Loss (ECL) Engine

**Project:** Credit Risk Scorecard + IFRS 9 Expected Credit Loss (ECL) Engine  
**Input:** `pd_predictions.csv` (from Notebook 2) + `cs-training-clean.csv` (from Notebook 1)  
**Objective:** Implement an IFRS 9-style ECL calculator using the PD model output

**Author:** *Your Name*  
**Date:** *2026*

---

### 📋 IFRS 9 ECL Formula

$$ECL = PD \times LGD \times EAD \times DF$$

| Term | Meaning | Source |
|------|---------|--------|
| **PD** | Probability of Default | Logistic Regression model (Notebook 2) |
| **LGD** | Loss Given Default | Assumed 45% (Basel II standard) |
| **EAD** | Exposure at Default | Derived from monthly income proxy |
| **DF** | Discount Factor | Based on effective interest rate |

### 📋 Notebook Contents
1. Load data & PD scores
2. Define LGD, EAD, Discount Factor
3. Calculate ECL (base scenario)
4. IFRS 9 Stage classification (Stage 1 / 2 / 3)
5. Scenario weighting (Base / Optimistic / Pessimistic)
6. ECL summary report
7. Export results

## Step 0: Upload Files (Google Colab Only)

In [ ]:
# Uncomment if running in Google Colab
# from google.colab import files
# uploaded = files.upload()  # Upload pd_predictions.csv AND cs-training-clean.csv

## Step 1: Import Libraries & Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.size'] = 12
sns.set_style('whitegrid')

print('Libraries loaded.')

In [ ]:
# ── Load PD predictions from Notebook 2 ─────────────────────
pd_df = pd.read_csv('pd_predictions.csv')

# ── Load full cleaned dataset for income data ────────────────
full_df = pd.read_csv('cs-training-clean.csv')

print(f'PD predictions : {pd_df.shape[0]:,} rows')
print(f'Full dataset   : {full_df.shape[0]:,} rows')
print()
print('PD predictions columns:')
print(list(pd_df.columns))
pd_df.head()

## Step 2: Define LGD, EAD, and Discount Factor

In a real bank, LGD and EAD are estimated from historical recovery data.  
For this project, we use industry-standard assumptions (clearly documented for reviewers).

In [ ]:
# ── IFRS 9 Parameters ────────────────────────────────────────

# LGD: Loss Given Default
# Basel II standard assumption for unsecured retail loans = 45%
LGD = 0.45

# Discount Factor (DF)
# Represents the time value of money over the expected loss period
# Using a standard effective interest rate of 5%
DISCOUNT_RATE = 0.05
DF = 1 / (1 + DISCOUNT_RATE)   # Single-period discount factor

# EAD: Exposure at Default
# Proxy: 12 months of monthly income (represents outstanding loan balance)
# We compute this per borrower using their monthly income
pd_df['EAD'] = pd_df['income_per_dependent'] * 12   # annual income proxy

# Cap EAD to avoid extreme outliers distorting the ECL totals
EAD_CAP = pd_df['EAD'].quantile(0.99)
pd_df['EAD'] = pd_df['EAD'].clip(upper=EAD_CAP)

print('=== IFRS 9 Parameter Assumptions ===')
print(f'  LGD           : {LGD*100:.0f}%  (Basel II unsecured retail standard)')
print(f'  Discount Rate : {DISCOUNT_RATE*100:.0f}%')
print(f'  Discount Factor: {DF:.4f}')
print(f'  EAD           : 12 x monthly income proxy (capped at 99th percentile = {EAD_CAP:,.0f})')
print()
print(f'Average EAD per borrower: {pd_df["EAD"].mean():,.0f}')

## Step 3: Calculate Base ECL

$$ECL_{base} = PD \times LGD \times EAD \times DF$$

In [ ]:
# ── Base ECL calculation ─────────────────────────────────────
pd_df['PD']       = pd_df['predicted_pd']
pd_df['LGD']      = LGD
pd_df['DF']       = DF
pd_df['ECL_base'] = pd_df['PD'] * pd_df['LGD'] * pd_df['EAD'] * pd_df['DF']

print('=== Base ECL Summary ===')
print(f'  Total borrowers       : {len(pd_df):,}')
print(f'  Total EAD             : {pd_df["EAD"].sum():>15,.0f}')
print(f'  Total Base ECL        : {pd_df["ECL_base"].sum():>15,.0f}')
print(f'  Average ECL per loan  : {pd_df["ECL_base"].mean():>15,.2f}')
print(f'  ECL / EAD ratio       : {pd_df["ECL_base"].sum() / pd_df["EAD"].sum() * 100:.2f}%')
print()
pd_df[['PD', 'LGD', 'EAD', 'DF', 'ECL_base']].describe().round(4)

## Step 4: IFRS 9 Stage Classification

Under IFRS 9, loans are classified into three stages based on credit risk:

| Stage | Criteria | ECL Measurement |
|-------|----------|-----------------|
| **Stage 1** | Low risk (PD < 10%) | 12-month ECL |
| **Stage 2** | Significant increase in credit risk (10% ≤ PD < 50%) | Lifetime ECL |
| **Stage 3** | Credit-impaired (PD ≥ 50% or already defaulted) | Lifetime ECL |

In [ ]:
# ── IFRS 9 Stage assignment ──────────────────────────────────
def assign_stage(row):
    if row['actual_default'] == 1 or row['PD'] >= 0.50:
        return 3   # Credit-impaired
    elif row['PD'] >= 0.10:
        return 2   # Significant increase in credit risk
    else:
        return 1   # Low risk

pd_df['Stage'] = pd_df.apply(assign_stage, axis=1)

# Stage multiplier: Stage 1 = 1 year ECL, Stage 2/3 = lifetime (3 years)
LIFETIME_YEARS = 3
pd_df['ECL_multiplier'] = pd_df['Stage'].map({1: 1, 2: LIFETIME_YEARS, 3: LIFETIME_YEARS})
pd_df['ECL_staged'] = pd_df['ECL_base'] * pd_df['ECL_multiplier']

# ── Stage summary ─────────────────────────────────────────────
stage_summary = pd_df.groupby('Stage').agg(
    Borrowers=('PD', 'count'),
    Avg_PD=('PD', 'mean'),
    Total_EAD=('EAD', 'sum'),
    Total_ECL=('ECL_staged', 'sum')
).round(4)

stage_summary['ECL_Rate'] = (stage_summary['Total_ECL'] / stage_summary['Total_EAD'] * 100).round(2).astype(str) + '%'
stage_summary['Avg_PD'] = (stage_summary['Avg_PD'] * 100).round(2).astype(str) + '%'

print('=== IFRS 9 Stage Classification Summary ===')
print(stage_summary.to_string())

In [ ]:
# ── Stage distribution chart ─────────────────────────────────
stage_counts = pd_df['Stage'].value_counts().sort_index()
stage_colors = ['#2ecc71', '#f39c12', '#e74c3c']
stage_labels = [f'Stage {i}' for i in stage_counts.index]

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Borrower count by stage
axes[0].bar(stage_labels, stage_counts.values, color=stage_colors, edgecolor='white', linewidth=1.5)
axes[0].set_title('Borrower Count by IFRS 9 Stage', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Number of Borrowers')
for i, v in enumerate(stage_counts.values):
    axes[0].text(i, v + 30, f'{v:,}\n({v/len(pd_df)*100:.1f}%)', ha='center', fontweight='bold', fontsize=10)

# ECL by stage
ecl_by_stage = pd_df.groupby('Stage')['ECL_staged'].sum()
axes[1].bar([f'Stage {i}' for i in ecl_by_stage.index], ecl_by_stage.values,
            color=stage_colors, edgecolor='white', linewidth=1.5)
axes[1].set_title('Total ECL by IFRS 9 Stage', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Total ECL')
axes[1].yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{x/1e6:.1f}M'))
for i, v in enumerate(ecl_by_stage.values):
    axes[1].text(i, v + ecl_by_stage.max()*0.02, f'{v/1e6:.2f}M', ha='center', fontweight='bold', fontsize=10)

plt.suptitle('IFRS 9 Stage Analysis', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig('plot_11_ifrs9_stages.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: plot_11_ifrs9_stages.png')

## Step 5: Scenario Weighting (Base / Optimistic / Pessimistic)

IFRS 9 requires ECL to be calculated as a **probability-weighted average** across multiple economic scenarios.

In [ ]:
# ── Scenario definitions ─────────────────────────────────────
# Each scenario adjusts PD by a multiplier to simulate different economic conditions

SCENARIOS = {
    'Base':        {'weight': 0.50, 'pd_multiplier': 1.00},   # 50% weight — current conditions
    'Optimistic':  {'weight': 0.25, 'pd_multiplier': 0.70},   # 25% weight — PD 30% lower (economic boom)
    'Pessimistic': {'weight': 0.25, 'pd_multiplier': 1.50},   # 25% weight — PD 50% higher (recession)
}

print('=== Economic Scenario Assumptions ===')
for name, params in SCENARIOS.items():
    print(f'  {name:12s} | Weight: {params["weight"]*100:.0f}%  | PD multiplier: {params["pd_multiplier"]}x')

print()
print(f'Total weight: {sum(s["weight"] for s in SCENARIOS.values())*100:.0f}%  ✓')

In [ ]:
# ── Calculate ECL for each scenario ─────────────────────────
scenario_results = {}

for scenario, params in SCENARIOS.items():
    # Adjust PD by scenario multiplier, cap at 100%
    pd_adj = (pd_df['PD'] * params['pd_multiplier']).clip(0, 1)
    ecl_scenario = pd_adj * LGD * pd_df['EAD'] * DF * pd_df['ECL_multiplier']
    scenario_results[scenario] = ecl_scenario
    pd_df[f'ECL_{scenario}'] = ecl_scenario

# ── Probability-weighted ECL ─────────────────────────────────
pd_df['ECL_weighted'] = sum(
    pd_df[f'ECL_{scenario}'] * params['weight']
    for scenario, params in SCENARIOS.items()
)

# ── Scenario comparison ──────────────────────────────────────
print('=== Scenario ECL Comparison ===')
print(f'{"Scenario":15s} {"Weight":>8s} {"Total ECL":>15s} {"ECL/EAD":>10s}')
print('-' * 52)

for scenario, params in SCENARIOS.items():
    total_ecl = pd_df[f'ECL_{scenario}'].sum()
    ecl_rate  = total_ecl / pd_df['EAD'].sum() * 100
    print(f'{scenario:15s} {params["weight"]*100:>7.0f}%  {total_ecl:>15,.0f}  {ecl_rate:>9.2f}%')

print('-' * 52)
weighted_total = pd_df['ECL_weighted'].sum()
weighted_rate  = weighted_total / pd_df['EAD'].sum() * 100
print(f'{"WEIGHTED TOTAL":15s} {"100%":>8s}  {weighted_total:>15,.0f}  {weighted_rate:>9.2f}%')

In [ ]:
# ── Scenario comparison chart ────────────────────────────────
scenario_totals = {
    s: pd_df[f'ECL_{s}'].sum() for s in SCENARIOS
}
scenario_totals['Weighted\nAverage'] = pd_df['ECL_weighted'].sum()

colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']
plt.figure(figsize=(10, 5))
bars = plt.bar(scenario_totals.keys(), scenario_totals.values(), color=colors, edgecolor='white', linewidth=1.5)
plt.title('Total ECL by Economic Scenario', fontsize=14, fontweight='bold')
plt.ylabel('Total ECL')
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{x/1e6:.1f}M'))
for bar in bars:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 1.02,
             f'{bar.get_height()/1e6:.2f}M', ha='center', fontweight='bold')
plt.tight_layout()
plt.savefig('plot_12_scenario_ecl.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: plot_12_scenario_ecl.png')

## Step 6: ECL Summary Report by Stage & Scenario

In [ ]:
# ── Full breakdown: Stage x Scenario ─────────────────────────
report = pd_df.groupby('Stage').agg(
    Borrowers      = ('PD', 'count'),
    Avg_PD         = ('PD', lambda x: f'{x.mean()*100:.2f}%'),
    Total_EAD      = ('EAD', lambda x: f'{x.sum():,.0f}'),
    ECL_Optimistic = ('ECL_Optimistic', lambda x: f'{x.sum():,.0f}'),
    ECL_Base       = ('ECL_Base', lambda x: f'{x.sum():,.0f}'),
    ECL_Pessimistic= ('ECL_Pessimistic', lambda x: f'{x.sum():,.0f}'),
    ECL_Weighted   = ('ECL_weighted', lambda x: f'{x.sum():,.0f}'),
)

print('=== IFRS 9 ECL Report — Stage x Scenario ===')
print(report.to_string())
print()

# Portfolio-level totals
print('=== Portfolio Totals ===')
print(f'  Borrowers         : {len(pd_df):,}')
print(f'  Total EAD         : {pd_df["EAD"].sum():>15,.0f}')
print(f'  ECL — Optimistic  : {pd_df["ECL_Optimistic"].sum():>15,.0f}')
print(f'  ECL — Base        : {pd_df["ECL_Base"].sum():>15,.0f}')
print(f'  ECL — Pessimistic : {pd_df["ECL_Pessimistic"].sum():>15,.0f}')
print(f'  ECL — Weighted    : {pd_df["ECL_weighted"].sum():>15,.0f}')
print(f'  ECL Coverage Ratio: {pd_df["ECL_weighted"].sum() / pd_df["EAD"].sum()*100:.2f}%')

In [ ]:
# ── PD distribution by stage ─────────────────────────────────
stage_colors = {1: '#2ecc71', 2: '#f39c12', 3: '#e74c3c'}

plt.figure(figsize=(10, 5))
for stage in [1, 2, 3]:
    subset = pd_df[pd_df['Stage'] == stage]['PD']
    plt.hist(subset, bins=40, alpha=0.6, color=stage_colors[stage],
             label=f'Stage {stage}  (n={len(subset):,})', density=True)

plt.axvline(x=0.10, color='grey', linestyle='--', linewidth=1, label='Stage 1→2 threshold (10%)')
plt.axvline(x=0.50, color='black', linestyle='--', linewidth=1, label='Stage 2→3 threshold (50%)')
plt.xlabel('Predicted PD')
plt.ylabel('Density')
plt.title('PD Distribution by IFRS 9 Stage', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.savefig('plot_13_pd_by_stage.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: plot_13_pd_by_stage.png')

## Step 7: Export Results

In [ ]:
# ── Save full ECL results ────────────────────────────────────
ecl_output = pd_df[[
    'PD', 'LGD', 'EAD', 'DF',
    'Stage',
    'ECL_base',
    'ECL_Optimistic', 'ECL_Base', 'ECL_Pessimistic',
    'ECL_weighted',
    'actual_default'
]].round(4)

ecl_output.to_csv('ecl_results.csv', index=False)
print('ECL results saved: ecl_results.csv')

# ── Save stage-level summary ──────────────────────────────────
stage_export = pd_df.groupby('Stage').agg(
    Borrowers       = ('PD', 'count'),
    Avg_PD          = ('PD', 'mean'),
    Total_EAD       = ('EAD', 'sum'),
    ECL_Optimistic  = ('ECL_Optimistic', 'sum'),
    ECL_Base        = ('ECL_Base', 'sum'),
    ECL_Pessimistic = ('ECL_Pessimistic', 'sum'),
    ECL_Weighted    = ('ECL_weighted', 'sum'),
).round(2)

stage_export.to_csv('ecl_stage_summary.csv')
print('Stage summary saved: ecl_stage_summary.csv')

print()
print('=== ECL Engine Complete ===')
print(f'  Model PD source   : Logistic Regression (AUC = 0.8611)')
print(f'  LGD assumption    : {LGD*100:.0f}% (Basel II standard)')
print(f'  Discount rate     : {DISCOUNT_RATE*100:.0f}%')
print(f'  Scenarios         : Optimistic / Base / Pessimistic')
print(f'  Weighted ECL      : {pd_df["ECL_weighted"].sum():,.0f}')
print(f'  ECL Coverage      : {pd_df["ECL_weighted"].sum()/pd_df["EAD"].sum()*100:.2f}%')
print()
print('Next step → Model Governance Pack & GitHub README')

---
## 📁 Output Files

| File | Description |
|------|-------------|
| `ecl_results.csv` | Full ECL results per borrower |
| `ecl_stage_summary.csv` | ECL summary by IFRS 9 Stage |
| `plot_11_ifrs9_stages.png` | Borrower count and ECL by stage |
| `plot_12_scenario_ecl.png` | ECL comparison across scenarios |
| `plot_13_pd_by_stage.png` | PD distribution by stage |

---
**Next Step:** Model Governance Pack — Model Card, Assumptions, Limitations & Monitoring Plan